<a href="https://colab.research.google.com/github/ss1705/Traffic-LLM/blob/main/XGBoost_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/TrafficLLM/congestion_data.csv')

In [4]:
df.head()

,ID,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,...,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,C-14344128,2,39.191032,-120.819740,2016-12-20T19:19:00.000-05:00,2016-12-20T19:33:47.000-05:00,1.40,2.58,2.60,Moderate,...,54.0,NaN,30.0,30.15,10.0,North,3.5,NaN,NaN,Clear
1,C-32285069,0,41.736015,-87.721565,2018-11-16T17:18:00.000-05:00,2018-11-16T18:08:28.000-05:00,0.73,0.42,1.00,Slow,...,39.0,31.8,70.0,30.06,10.0,West,11.5,NaN,NaN,Overcast
2,C-14213642,0,32.519043,-93.741096,2021-02-18T20:32:00.000-05:00,2021-02-18T21:21:32.000-05:00,1.80,1.00,2.00,Moderate,...,30.0,30.0,79.0,30.20,10.0,WNW,3.0,0.00,NaN,Fair
3,C-29674072,0,40.730564,-74.001709,2020-11-13T08:06:00.000-05:00,2020-11-13T08:48:22.000-05:00,1.42,1.00,2.00,Slow,...,48.0,48.0,93.0,29.92,1.0,VAR,3.0,0.04,NaN,Light Rain
4,C-24044478,1,33.758331,-118.238533,2017-08-24T09:54:00.000-04:00,2017-08-24T11:13:19.000-04:00,2.60,4.90,6.92,Slow,...,66.9,NaN,79.0,29.89,9.0,Calm,NaN,NaN,NaN,Overcast


In [5]:
len(df)

2000000

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, accuracy_score

In [36]:
df.head(10)

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,2,39.191032,-120.819740,27350400.0,27349070.0,1.40,2.58,2.60,1,54.0,NaN,30.0,30.15,10.00,11,3.5,NaN,24,7
1,0,41.736015,-87.721565,87477540.0,87478351.0,0.73,0.42,1.00,2,39.0,31.8,70.0,30.06,10.00,23,11.5,NaN,24,88
2,0,32.519043,-93.741096,158769180.0,158769935.0,1.80,1.00,2.00,1,30.0,30.0,79.0,30.20,10.00,21,3.0,0.00,24,14
3,0,40.730564,-74.001709,150343620.0,150343945.0,1.42,1.00,2.00,2,48.0,48.0,93.0,29.92,1.00,18,3.0,0.04,24,59
4,1,33.758331,-118.238533,48653700.0,48656242.0,2.60,4.90,6.92,2,66.9,NaN,79.0,29.89,9.00,1,NaN,NaN,24,88
5,1,42.665081,-85.661926,190216260.0,190216601.0,6.88,3.00,3.00,0,21.0,8.0,92.0,28.88,0.75,8,15.0,0.04,24,68
6,2,40.766750,-73.950722,149491380.0,149492541.0,2.39,2.00,5.00,2,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,24,138
7,1,40.705330,-74.089493,156156180.0,156156725.0,4.93,3.00,5.00,1,40.0,32.0,49.0,29.93,10.00,16,14.0,0.00,24,85
8,1,33.467129,-112.169113,95689260.0,95689747.0,1.15,0.83,1.60,2,57.2,NaN,23.0,30.00,10.00,15,17.3,NaN,24,102
9,2,34.163628,-118.295052,49395000.0,49395232.0,3.76,3.60,3.65,0,93.9,NaN,26.0,29.73,10.00,22,5.8,NaN,24,90


In [8]:
df = df.drop(["Description", "Street", "City", "County", "State", "Country", "ZipCode", "LocalTimeZone",
               "WeatherStation_AirportCode", "WeatherTimeStamp","ID"], axis=1, errors="ignore")

In [9]:
df.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,2,39.191032,-120.819740,2016-12-20T19:19:00.000-05:00,2016-12-20T19:33:47.000-05:00,1.40,2.58,2.60,Moderate,54.0,NaN,30.0,30.15,10.0,North,3.5,NaN,NaN,Clear
1,0,41.736015,-87.721565,2018-11-16T17:18:00.000-05:00,2018-11-16T18:08:28.000-05:00,0.73,0.42,1.00,Slow,39.0,31.8,70.0,30.06,10.0,West,11.5,NaN,NaN,Overcast
2,0,32.519043,-93.741096,2021-02-18T20:32:00.000-05:00,2021-02-18T21:21:32.000-05:00,1.80,1.00,2.00,Moderate,30.0,30.0,79.0,30.20,10.0,WNW,3.0,0.00,NaN,Fair
3,0,40.730564,-74.001709,2020-11-13T08:06:00.000-05:00,2020-11-13T08:48:22.000-05:00,1.42,1.00,2.00,Slow,48.0,48.0,93.0,29.92,1.0,VAR,3.0,0.04,NaN,Light Rain
4,1,33.758331,-118.238533,2017-08-24T09:54:00.000-04:00,2017-08-24T11:13:19.000-04:00,2.60,4.90,6.92,Slow,66.9,NaN,79.0,29.89,9.0,Calm,NaN,NaN,NaN,Overcast


In [10]:
categorical_cols = ["Weather_Event", "Weather_Conditions", "WindDir","Congestion_Speed"]
for col in categorical_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

In [11]:
df.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,2,39.191032,-120.819740,2016-12-20T19:19:00.000-05:00,2016-12-20T19:33:47.000-05:00,1.40,2.58,2.60,1,54.0,NaN,30.0,30.15,10.0,11,3.5,NaN,24,7
1,0,41.736015,-87.721565,2018-11-16T17:18:00.000-05:00,2018-11-16T18:08:28.000-05:00,0.73,0.42,1.00,2,39.0,31.8,70.0,30.06,10.0,23,11.5,NaN,24,88
2,0,32.519043,-93.741096,2021-02-18T20:32:00.000-05:00,2021-02-18T21:21:32.000-05:00,1.80,1.00,2.00,1,30.0,30.0,79.0,30.20,10.0,21,3.0,0.00,24,14
3,0,40.730564,-74.001709,2020-11-13T08:06:00.000-05:00,2020-11-13T08:48:22.000-05:00,1.42,1.00,2.00,2,48.0,48.0,93.0,29.92,1.0,18,3.0,0.04,24,59
4,1,33.758331,-118.238533,2017-08-24T09:54:00.000-04:00,2017-08-24T11:13:19.000-04:00,2.60,4.90,6.92,2,66.9,NaN,79.0,29.89,9.0,1,NaN,NaN,24,88


In [12]:
df["StartTime"] = pd.to_datetime(df["StartTime"], utc=True, errors="coerce")
df["EndTime"] = pd.to_datetime(df["EndTime"], utc=True, errors="coerce")

In [13]:
df = df.dropna(subset=["StartTime", "EndTime"])

In [14]:
df["StartTime"] = (df["StartTime"] - df["StartTime"].min()).dt.total_seconds()
df["EndTime"] = (df["EndTime"] - df["EndTime"].min()).dt.total_seconds()

<ipython-input-14-393a4858df27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["StartTime"] = (df["StartTime"] - df["StartTime"].min()).dt.total_seconds()


In [15]:
df.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,2,39.191032,-120.819740,27350400.0,27349070.0,1.40,2.58,2.60,1,54.0,NaN,30.0,30.15,10.0,11,3.5,NaN,24,7
1,0,41.736015,-87.721565,87477540.0,87478351.0,0.73,0.42,1.00,2,39.0,31.8,70.0,30.06,10.0,23,11.5,NaN,24,88
2,0,32.519043,-93.741096,158769180.0,158769935.0,1.80,1.00,2.00,1,30.0,30.0,79.0,30.20,10.0,21,3.0,0.00,24,14
3,0,40.730564,-74.001709,150343620.0,150343945.0,1.42,1.00,2.00,2,48.0,48.0,93.0,29.92,1.0,18,3.0,0.04,24,59
4,1,33.758331,-118.238533,48653700.0,48656242.0,2.60,4.90,6.92,2,66.9,NaN,79.0,29.89,9.0,1,NaN,NaN,24,88


In [16]:
X = df.drop(["DelayFromTypicalTraffic(mins)", "Congestion_Speed"], axis=1)
y_regression = df["DelayFromTypicalTraffic(mins)"]  # Regression target
y_classification = df["Congestion_Speed"]  # Binary Classification (0 = Fast, 1 = Moderate, 2 = Slow)

In [17]:
X_train, X_test, y_reg_train, y_reg_test, y_cls_train, y_cls_test = train_test_split(
    X, y_regression, y_classification, test_size=0.2, random_state=42
)

In [18]:
model = xgb.XGBRegressor(objective="reg:squarederror", enable_categorical=True)
clf = xgb.XGBClassifier(objective="binary:logistic", enable_categorical=True)

In [19]:
model.fit(X_train, y_reg_train)
clf.fit(X_train, y_cls_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [30]:
X_test.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromFreeFlowSpeed(mins),Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
1561176,0,40.683048,-73.973801,196858260.0,196858583.0,0.72,2.00,69.0,69.0,39.0,29.92,10.0,0,0.0,0.0,24,14
1664099,2,36.685207,-114.509712,46640520.0,46641018.0,0.48,1.42,94.6,NaN,23.0,29.80,10.0,9,5.8,NaN,24,90
1239935,0,41.681252,-72.944443,123680220.0,123681522.0,1.34,3.00,26.0,NaN,58.0,30.61,10.0,24,NaN,0.0,24,14
1686969,1,28.885094,-81.687233,47449500.0,47449855.0,3.65,3.93,91.0,91.0,57.0,30.04,10.0,18,6.0,0.0,24,14
988622,1,33.856529,-117.974197,150637740.0,150638034.0,1.05,1.00,90.0,90.0,12.0,29.80,10.0,15,12.0,0.0,24,14


In [20]:
y_reg_pred = model.predict(X_test)
y_cls_pred = clf.predict(X_test)

In [21]:
mae = mean_absolute_error(y_reg_test, y_reg_pred)
accuracy = accuracy_score(y_cls_test, y_cls_pred)

print(f"Mean Absolute Error (MAE) for Delay Prediction: {mae}")
print(f"Accuracy for Congestion Prediction: {accuracy}")

Mean Absolute Error (MAE) for Delay Prediction: 0.7487718644020829
Accuracy for Congestion Prediction: 0.7246844628612037


In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_cls_test, y_cls_pred))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77    143084
           1       0.67      0.74      0.70    159343
           2       0.71      0.68      0.70     97209

    accuracy                           0.72    399636
   macro avg       0.73      0.72      0.72    399636
weighted avg       0.73      0.72      0.73    399636



In [23]:
importance = model.feature_importances_
for feature, score in zip(X_train.columns, importance):
    print(f"{feature}: {score:.4f}")

Severity: 0.0288
Start_Lat: 0.0275
Start_Lng: 0.0342
StartTime: 0.0393
EndTime: 0.0543
Distance(mi): 0.0496
DelayFromFreeFlowSpeed(mins): 0.5441
Temperature(F): 0.0516
WindChill(F): 0.0228
Humidity(%): 0.0216
Pressure(in): 0.0230
Visibility(mi): 0.0209
WindDir: 0.0156
WindSpeed(mph): 0.0116
Precipitation(in): 0.0163
Weather_Event: 0.0257
Weather_Conditions: 0.0131


In [24]:
y_cls_train.value_counts(normalize=True)

,proportion
Congestion_Speed,
1,0.399171
0,0.358740
2,0.242089


In [25]:
df["Congestion_Speed"].unique()

array([1, 2, 0])

In [26]:
df["Congestion_Speed"].value_counts()

,count
Congestion_Speed,
1,797434
0,716545
2,484198


In [31]:
new_sample = {
    "Severity": [2],
    "Start_Lat": [39.191032],
    "Start_Lng": [-120.819740],
    "StartTime": [27350400.0],   # Ensure it's in the same transformed format
    "EndTime": [27349070.0],     # Ensure it's in the same transformed format
    "Distance(mi)": [1.40],
    "DelayFromFreeFlowSpeed(mins)": [2.60],  # Keep this since it was important
    "Temperature(F)": [54.0],
    "WindChill(F)": [69.0],  # Handle missing values if needed
    "Humidity(%)": [30.0],
    "Pressure(in)": [30.15],
    "Visibility(mi)": [10.0],
    "WindDir": [11],  # If encoded, use the correct encoded value
    "WindSpeed(mph)": [3.5],
    "Precipitation(in)": [0.0],  # Handle missing values
    "Weather_Event": [24],  # Ensure encoding matches training data
    "Weather_Conditions": [7]  # Ensure encoding matches training data
}

In [32]:
sample_df = pd.DataFrame(new_sample)

In [33]:
predicted_delay = model.predict(sample_df)

In [34]:
predicted_congestion = clf.predict(sample_df)

In [35]:
print(f"Predicted Delay: {predicted_delay[0]:.2f} minutes")
print(f"Predicted Congestion Level: {predicted_congestion[0]}") # 0 -> High

Predicted Delay: 2.32 minutes
Predicted Congestion Level: 0
